In [ ]:
# 1. 표준 라이브러리
import sys
from pathlib import Path
import platform
import time
import os
import json

# 2. 서드파티 라이브러리 

# 2-1. 시각화
import plotly.express as px
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.font_manager as fm
import seaborn as sns

# 2-2. 
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.impute import SimpleImputer
from lightgbm import LGBMClassifier
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import StandardScaler, label_binarize
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, classification_report, balanced_accuracy_score, roc_auc_score

sys.path.append(str(Path.cwd().parent))
from utils import DATA_DIR, MODEL_DIR

In [2]:
# 한글 폰트 설정
if platform.system() == 'Windows':
    plt.rcParams['font.family'] = 'Malgun Gothic'
elif platform.system() == 'Darwin':  # macOS
    plt.rcParams['font.family'] = 'AppleGothic'
else:  # Linux
    plt.rcParams['font.family'] = 'NanumGothic'

plt.rcParams['axes.unicode_minus'] = False

mpl.rcParams['axes.unicode_minus'] = False
# 폰트 개인 경로에 맞춰서 변경
# FONT_DIR = Path("/path/to/fonts")
# font_path = FONT_DIR / 'FREESENTATION-6SEMIBOLD.ttf'
# prop = fm.FontProperties(fname=font_path)

In [3]:
pandas_kwargs = {
    'parse_dates': ['검정일자'],
    'date_format': '%Y-%m-%d'
}

milk: pd.DataFrame = pd.read_csv(DATA_DIR /'interim' / 'milk.csv', **pandas_kwargs)
milk.head()

,농장아이디,개체번호,검정일자,누적착유일(연계),유지방율,유단백,무지고형분,체세포수,MUN,305일유량,...,공태일수,유량,질병군,우유등급,비유초기평균MUN.1,비유최고도달일수_log,건유전마지막유량_log,공태일수_log,출생년도,PF비
0,21133,2.007121e+13,2021-03-18,445,6.60,3.28,8.87,274,11.6,15158.0,...,619,76169939,0,2등급이하,NaN,4.143135,4.330733,6.429719,2007,0.496970
1,21133,2.007121e+13,2021-04-28,486,3.64,3.33,8.91,166,11.0,15158.0,...,619,76170244,0,1등급,NaN,4.143135,4.330733,6.429719,2007,0.914835
2,21133,2.007121e+13,2021-05-20,508,2.61,3.35,9.14,156,10.8,15158.0,...,619,76170487,0,1등급,NaN,4.143135,4.330733,6.429719,2007,1.283525
3,21133,2.007121e+13,2021-06-25,544,3.75,3.49,8.99,307,14.1,15158.0,...,619,76170734,0,2등급이하,NaN,4.143135,4.330733,6.429719,2007,0.930667
4,21133,2.007121e+13,2021-07-07,556,3.68,3.74,9.31,34,16.2,15158.0,...,619,76170903,0,1등급,NaN,4.143135,4.330733,6.429719,2007,1.016304


In [4]:
df = milk

In [5]:
df['1등급'] = df['우유등급'].map({
    '1등급': 0,
    '2등급이하': 1
})

In [6]:
target='1등급'
features = [
    '누적착유일(연계)',
    '전산차비유지속성',
    '현재산차비유지속성',
    '비유최고도달일수_log',
    '건유전마지막유량_log',
    '전산차건유전유량',
    '산차',
    '농후사료급여량(연계)',
    '공태일수_log'
]

In [7]:
train = df[df['검정일자'].dt.year == 2020]
test = df[df['검정일자'].dt.year == 2021]

X_train, X_test = train[features], test[features]
y_train, y_test = train['1등급'], test['1등급']

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# ============================================
# 1. 클래스 비율 확인
# ============================================
print("="*60)
print("클래스 분포 분석")
print("="*60)
print(y_train.value_counts().sort_index())
print(f"\n비율:")
print(y_train.value_counts(normalize=True).sort_index())

# ✅ 수정: 0이 1등급(다수), 1이 그외(소수)
majority_count = (y_train == 0).sum()   # 1등급 (다수) - 약 90%
minority_count = (y_train == 1).sum()   # 그외 (소수) - 약 10%

print(f"\n클래스 구성:")
print(f"   0 (1등급, 다수): {majority_count}개 ({majority_count/len(y_train)*100:.1f}%)")
print(f"   1 (그외, 소수): {minority_count}개 ({minority_count/len(y_train)*100:.1f}%)")

# ✅ scale_pos_weight: 양성 클래스(1=그외=소수)에 대한 가중치
# 공식: (다수 클래스 개수) / (소수 클래스 개수)
scale_pos_weight = majority_count / minority_count

print(f"\n계산된 scale_pos_weight: {scale_pos_weight:.2f}")
print(f"   → LightGBM은 자동으로 클래스 1(그외)에 이 가중치 적용")

# 결측치 확인
print("\n결측치 현황:")
missing_info = pd.DataFrame({
    'Feature': X_train.columns,
    '결측_개수': X_train.isnull().sum().values,
    '결측_비율(%)': (X_train.isnull().mean() * 100).values
}).sort_values('결측_비율(%)', ascending=False)
print(missing_info.head(10))
print(f"\n전체 결측 비율: {X_train.isnull().mean().mean()*100:.1f}%")
print(f"Feature 개수: {X_train.shape[1]}개")

# ============================================
# 2. Pipeline 구성
# ============================================
pipeline = ImbPipeline([    
    ('model', LGBMClassifier(
        objective='binary',
        device='gpu',
        random_state=42,
        verbosity=-1,
        n_jobs=1,
        scale_pos_weight=scale_pos_weight,  # ✅ 자동으로 pos_label=1에 적용
        force_col_wise=True
    ))
])

print("\n✅ Pipeline 구성:")
print("   1. LGBMClassifier (GPU)")
print(f"      - scale_pos_weight={scale_pos_weight:.2f} (클래스 1=그외 가중치)")

# ============================================
# 3. 파라미터 분포
# ============================================
print(f"\n소수 클래스(1=그외) 샘플 수: {minority_count}개")

from scipy.stats import uniform, randint

param_distributions = {
    # 🔥 LightGBM 파라미터 - 소수 클래스 민감도 향상
    'model__n_estimators': randint(300, 1000),
    'model__learning_rate': uniform(0.005, 0.045),
    'model__max_depth': randint(3, 8),
    'model__num_leaves': randint(15, 80),
    'model__min_child_samples': randint(5, 50),
    'model__subsample': uniform(0.6, 0.4),
    'model__colsample_bytree': uniform(0.6, 0.4),
    'model__min_child_weight': [1e-5, 1e-4, 1e-3, 1e-2],
    'model__reg_alpha': uniform(0, 0.5),
    'model__reg_lambda': uniform(0, 0.5),
}

# ============================================
# 4. Scoring 설정
# ============================================
from sklearn.metrics import make_scorer, recall_score, precision_score, f1_score, fbeta_score, average_precision_score

# ✅ 이제 pos_label=1이 기본값이므로 생략 가능!
# 하지만 명확성을 위해 명시하는 것 추천 (특히 팀 협업 시)

scoring = {
    'pr_auc': make_scorer(average_precision_score, needs_proba=True),  # ✅ pos_label=1 기본
    'recall': make_scorer(recall_score),                               # ✅ pos_label=1 기본
    'f2': make_scorer(fbeta_score, beta=2),                            # ✅ pos_label=1 기본
    'precision': make_scorer(precision_score, zero_division=0),        # ✅ pos_label=1 기본
    'f1': make_scorer(f1_score, zero_division=0),                      # ✅ pos_label=1 기본
    'roc_auc': 'roc_auc'                                               # ✅ pos_label=1 기본
}

main_score = 'pr_auc'

print("\n✅ Scoring 전략:")
print(f"   메인 목표: {main_score.upper()}")
print(f"   타깃: 클래스 1 (그외) 검출 최적화")
print(f"   pos_label: 1 (기본값, 생략 가능)")

# ============================================
# 5. Stratified K-Fold
# ============================================
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
n_folds = 5
n_iter = 150

print(f"\n교차검증 Folds: {n_folds}")
print(f"랜덤 샘플링 조합 수: {n_iter}개")
print(f"총 Fits: {n_iter} × {n_folds} = {n_iter * n_folds}")

# ============================================
# 6. RandomizedSearchCV 실행
# ============================================
print("\n" + "="*60)
print("🚀 RandomizedSearchCV 시작!")
print("="*60)

lgbm_random = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_distributions,
    n_iter=n_iter,
    cv=cv,
    scoring=scoring,
    refit=main_score,
    verbose=2,
    random_state=42,
    n_jobs=-1,
)

start_actual = time.time()
lgbm_random.fit(X_train, y_train)
actual_time = time.time() - start_actual

# ============================================
# 7. 결과 출력
# ============================================
print("\n" + "="*60)
print("✅ 완료!")
print("="*60)
print(f"⏱️  소요 시간: {actual_time/60:.1f}분 ({actual_time:.0f}초)")

print(f"\n🏆 최고 {main_score.upper()} (CV): {lgbm_random.best_score_:.4f}")

print("\n📋 최적 파라미터:")
for key, value in sorted(lgbm_random.best_params_.items()):
    if isinstance(value, float):
        print(f"   {key}: {value:.4f}")
    else:
        print(f"   {key}: {value}")

# ============================================
# 8. 전체 결과 비교
# ============================================
results_df = pd.DataFrame(lgbm_random.cv_results_)

comparison_cols = [
    f'rank_test_{main_score}',
    f'mean_test_{main_score}',
    'mean_test_recall',
    'mean_test_f2',
    'mean_test_precision',
    'mean_test_f1',
    'mean_test_roc_auc',
    'params'
]

print("\n" + "="*60)
print(f"📊 상위 10개 파라미터 조합 ({main_score.upper()} 기준)")
print("="*60)
top_10 = results_df[comparison_cols].sort_values(f'rank_test_{main_score}').head(10)

for idx, row in top_10.iterrows():
    rank = int(row[f'rank_test_{main_score}'])
    pr_auc = row[f'mean_test_{main_score}']
    recall = row['mean_test_recall']
    f2 = row['mean_test_f2']
    precision = row['mean_test_precision']
    f1 = row['mean_test_f1']
    
    print(f"\n#{rank} - {main_score.upper()}: {pr_auc:.4f} | Recall: {recall:.4f}")
    print(f"   F2: {f2:.3f} | Precision: {precision:.3f} | F1: {f1:.3f}")
    
    params = row['params']
    print(f"   Model: lr={params.get('model__learning_rate', 0):.4f}, "
          f"depth={params.get('model__max_depth', 0)}, "
          f"leaves={params.get('model__num_leaves', 0)}")

# ============================================
# 9. 테스트 세트 평가
# ============================================
print("\n" + "="*60)
print("🧪 테스트 세트 평가")
print("="*60)

best_model = lgbm_random.best_estimator_
y_pred = best_model.predict(X_test)
y_pred_proba = best_model.predict_proba(X_test)

# ✅ 클래스 1(그외)의 확률 추출
y_pred_proba_minority = y_pred_proba[:, 1]  # ✅ 인덱스 1 = 클래스 1 = 그외

print(classification_report(
    y_test, y_pred, 
    target_names=['0 (1등급, 다수)', '1 (그외, 소수)']
))

print("\n혼동 행렬:")
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(cm,
                     index=['실제_0 (1등급)', '실제_1 (그외)'],
                     columns=['예측_0 (1등급)', '예측_1 (그외)'])
print(cm_df)

# ✅ 혼동 행렬 해석 수정
# confusion_matrix 구조:
#              예측_0  예측_1
# 실제_0 (1등급)  TN      FP
# 실제_1 (그외)   FN      TP

tn, fp, fn, tp = cm.ravel()

print("\n💼 비즈니스 해석 (소수 클래스=1=그외 기준):")
print(f"   ✅ 정확한 분류: {tn + tp}개 ({(tn+tp)/len(y_test)*100:.1f}%)")
print(f"   ⚠️  오탐 (FP): {fp}개 → 1등급을 그외로 오판 (재검사 필요)")
print(f"   🚨 미탐 (FN): {fn}개 → 그외를 1등급으로 오판 (치명적!)")

# ✅ Recall 계산: TP / (TP + FN)
if tp + fn > 0:
    recall_pct = tp/(tp+fn)*100
    print(f"\n   📊 소수 클래스(1=그외) Recall: {recall_pct:.1f}%")
    print(f"      → 그외 {tp+fn}개 중 {tp}개 검출 ({fn}개 놓침)")
else:
    print(f"\n   📊 소수 클래스(1=그외) Recall: N/A")

# ✅ Precision 계산: TP / (TP + FP)
if tp + fp > 0:
    precision_pct = tp/(tp+fp)*100
    print(f"   📊 소수 클래스(1=그외) Precision: {precision_pct:.1f}%")
    print(f"      → 그외 판정 {tp+fp}개 중 {tp}개 정답")
else:
    print(f"   📊 소수 클래스(1=그외) Precision: N/A")

test_roc_auc = roc_auc_score(y_test, y_pred_proba_minority)
print(f"   📊 ROC-AUC: {test_roc_auc:.4f}")

# ============================================
# 10. 🔥 임계값 조정으로 추가 개선
# ============================================
from sklearn.metrics import precision_recall_curve, average_precision_score

# ✅ pos_label=1 (그외) 기준으로 PR 곡선 계산
precisions, recalls, thresholds = precision_recall_curve(
    y_test, y_pred_proba_minority, pos_label=1
)

# Recall 80% 보장
target_recall = 0.80
idx = np.where(recalls >= target_recall)[0]

if len(idx) > 0:
    best_idx = idx[np.argmax(precisions[idx])]
    optimal_threshold = thresholds[best_idx]
    
    print("\n" + "="*60)
    print(f"🎯 Recall {target_recall*100}% 보장 시 최적 임계값")
    print("="*60)
    print(f"최적 임계값: {optimal_threshold:.3f} (기본값 0.5)")
    print(f"Precision: {precisions[best_idx]:.3f}")
    print(f"Recall: {recalls[best_idx]:.3f}")
    
    # ✅ 임계값 조정: 확률 >= threshold이면 클래스 1(그외)로 예측
    y_pred_adjusted = (y_pred_proba_minority >= optimal_threshold).astype(int)
    
    cm_adj = confusion_matrix(y_test, y_pred_adjusted)
    tn2, fp2, fn2, tp2 = cm_adj.ravel()
    
    print(f"\n조정 효과:")
    print(f"   미탐(FN) 감소: {fn} → {fn2} (개선: {fn-fn2}개)")
    print(f"   오탐(FP) 증가: {fp} → {fp2} (증가: {fp2-fp}개)")
    
    if tp + fn > 0 and tp2 + fn2 > 0:
        print(f"   Recall 개선: {tp/(tp+fn)*100:.1f}% → {tp2/(tp2+fn2)*100:.1f}%")
    
    if fn > fn2:
        print(f"   💡 그외를 1등급으로 오판하는 치명적 오류 {fn-fn2}개 감소!")
else:
    print(f"\n⚠️  Recall {target_recall*100}% 달성 불가 (최대 Recall: {recalls.max():.2%})")

# ============================================
# 11. 변수 중요도 확인
# ============================================
print("\n" + "="*60)
print("📊 변수 중요도 Top 10")
print("="*60)

final_model = best_model.named_steps['model']
feature_names = list(X_train.columns)

n_features_used = len(final_model.feature_importances_)
feature_names = feature_names[:n_features_used]

importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': final_model.feature_importances_
}).sort_values('importance', ascending=False)

print(importance_df.head(10).to_string(index=False))

# ============================================
# 12. 교차검증 vs 테스트 성능 비교
# ============================================
print("\n" + "="*60)
print("📈 성능 비교: 교차검증 vs 테스트")
print("="*60)

# ✅ pos_label=1 기본값 사용
test_metrics = {
    'PR-AUC': average_precision_score(y_test, y_pred_proba_minority),
    'Recall': recall_score(y_test, y_pred),
    'F2': fbeta_score(y_test, y_pred, beta=2),
    'Precision': precision_score(y_test, y_pred, zero_division=0),
    'F1': f1_score(y_test, y_pred, zero_division=0),
    'ROC-AUC': test_roc_auc
}

print(f"{'지표':<15} {'교차검증 (CV)':<20} {'테스트':<15} {'차이':<10}")
print("-" * 60)

for metric in ['PR-AUC', 'Recall', 'F2', 'Precision', 'F1', 'ROC-AUC']:
    cv_key = metric.lower().replace('-', '_')
    cv_score = results_df.loc[results_df[f'rank_test_{main_score}'] == 1, f'mean_test_{cv_key}'].values[0]
    test_score = test_metrics[metric]
    diff = test_score - cv_score
    
    diff_str = f"{diff:+.4f}"
    if abs(diff) > 0.05:
        diff_str += " ⚠️"
    
    print(f"{metric:<15} {cv_score:<20.4f} {test_score:<15.4f} {diff_str}")

# ============================================
# 13. 최종 요약
# ============================================
print("\n" + "="*60)
print(f"📝 최종 요약 ({main_score.upper()} 최적화)")
print("="*60)
print(f"✅ 최적 모델 {main_score.upper()} (CV): {lgbm_random.best_score_:.4f}")
print(f"✅ 테스트 Recall: {test_metrics['Recall']*100:.1f}% (그외 검출률)")
print(f"✅ 테스트 Precision: {test_metrics['Precision']*100:.1f}% (그외 판정 정확도)")
print(f"✅ 테스트 F1: {test_metrics['F1']:.4f}")
print(f"✅ 치명적 오류(미탐 FN): {fn}개 / {tp+fn}개")
print(f"⏱️  총 소요 시간: {actual_time/60:.1f}분")

print("\n💡 핵심 포인트:")
print(f"   - 그외(1) {tp+fn}개 중 {tp}개 검출 ({tp/(tp+fn)*100:.1f}%)")
print(f"   - 놓친 그외: {fn}개")
print(f"   - 오탐 (1등급→그외): {fp}개 (재검사 필요)")

# ============================================
# 14. 모델 + 파라미터 저장
# ============================================
print("\n" + "="*60)
print("💾 모델 및 파라미터 저장")
print("="*60)

save_model_input = input("모델을 저장하시겠습니까? (y/n): ")
if save_model_input.lower() == 'y':
    import joblib
    from datetime import datetime
    from pathlib import Path
    
    # MODEL_DIR이 정의되지 않은 경우 현재 디렉토리 사용
    try:
        MODEL_DIR = Path(MODEL_DIR)
    except:
        MODEL_DIR = Path('.')
        print(f"⚠️  MODEL_DIR 미정의, 현재 디렉토리 사용: {MODEL_DIR.absolute()}")
    
    MODEL_DIR.mkdir(exist_ok=True)
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    model_filename = MODEL_DIR / f'best_lgbm_{main_score}_optimized_{timestamp}.pkl'
    params_filename = MODEL_DIR / f'best_params_{main_score}_{timestamp}.json'
    metrics_filename = MODEL_DIR / f'test_metrics_{main_score}_{timestamp}.json'
    results_filename = MODEL_DIR / f'cv_results_{main_score}_{timestamp}.csv'
    
    # 1. 모델 저장
    joblib.dump(best_model, model_filename)
    print(f"✅ 모델 저장: {model_filename}")
    
    # 2. 파라미터 저장
    best_params_serializable = {}
    for key, value in lgbm_random.best_params_.items():
        if isinstance(value, (np.integer, np.floating)):
            best_params_serializable[key] = float(value) if isinstance(value, np.floating) else int(value)
        else:
            best_params_serializable[key] = value
    
    params_info = {
        'best_params': best_params_serializable,
        f'best_{main_score}_score': float(lgbm_random.best_score_),
        'optimization_target': main_score,
        'cv_folds': n_folds,
        'n_iter': n_iter,
        'timestamp': timestamp,
        'class_labels': {
            '0': '1등급 (다수)',
            '1': '그외 (소수)'
        },
        'minority_count': int(minority_count),
        'majority_count': int(majority_count),
        'scale_pos_weight': float(scale_pos_weight)
    }
    
    with open(params_filename, 'w', encoding='utf-8') as f:
        json.dump(params_info, f, indent=2, ensure_ascii=False)
    print(f"✅ 파라미터 저장: {params_filename}")
    
    # 3. 테스트 성능 저장
    test_metrics_serializable = {
        key: float(value) if isinstance(value, (np.floating, np.integer)) else value
        for key, value in test_metrics.items()
    }
    
    metrics_info = {
        'test_metrics': test_metrics_serializable,
        'confusion_matrix': {
            'tn': int(tn),  # 1등급 → 1등급 (정답)
            'fp': int(fp),  # 1등급 → 그외 (오탐)
            'fn': int(fn),  # 그외 → 1등급 (미탐, 치명적!)
            'tp': int(tp)   # 그외 → 그외 (정답)
        },
        'business_interpretation': {
            'total_minority': int(tp + fn),      # 실제 그외 개수
            'detected_minority': int(tp),        # 검출된 그외
            'missed_minority': int(fn),          # 놓친 그외 (치명적)
            'recall_percentage': float(tp/(tp+fn)*100) if (tp+fn) > 0 else 0,
            'false_alarms': int(fp)              # 1등급을 그외로 오판
        },
        'timestamp': timestamp
    }
    
    with open(metrics_filename, 'w', encoding='utf-8') as f:
        json.dump(metrics_info, f, indent=2, ensure_ascii=False)
    print(f"✅ 테스트 성능 저장: {metrics_filename}")
    
    # 4. CV 결과 저장
    results_df.to_csv(results_filename, index=False, encoding='utf-8')
    print(f"✅ CV 결과 저장: {results_filename}")
    
    print("\n📦 저장된 파일 요약:")
    print(f"1. 모델: {model_filename.name}")
    print(f"2. 파라미터: {params_filename.name}")
    print(f"3. 성능: {metrics_filename.name}")
    print(f"4. CV결과: {results_filename.name}")
    
else:
    print("모델 저장 생략")

print("\n" + "="*60)
print("🎉 모든 작업 완료!")
print("="*60)

클래스 분포 분석
1등급
0    26393
1     3682
Name: count, dtype: int64

비율:
1등급
0    0.877573
1    0.122427
Name: proportion, dtype: float64

클래스 구성:
   0 (1등급, 다수): 26393개 (87.8%)
   1 (그외, 소수): 3682개 (12.2%)

계산된 scale_pos_weight: 7.17
   → LightGBM은 자동으로 클래스 1(그외)에 이 가중치 적용

결측치 현황:
        Feature  결측_개수   결측_비율(%)
4  건유전마지막유량_log  13887  46.174564
5      전산차건유전유량  13887  46.174564
6            산차   2822   9.383209
0     누적착유일(연계)      0   0.000000
1      전산차비유지속성      0   0.000000
3  비유최고도달일수_log      0   0.000000
2     현재산차비유지속성      0   0.000000
7   농후사료급여량(연계)      0   0.000000
8      공태일수_log      0   0.000000

전체 결측 비율: 11.3%
Feature 개수: 9개

✅ Pipeline 구성:
   1. LGBMClassifier (GPU)
      - scale_pos_weight=7.17 (클래스 1=그외 가중치)

소수 클래스(1=그외) 샘플 수: 3682개

✅ Scoring 전략:
   메인 목표: PR_AUC
   타깃: 클래스 1 (그외) 검출 최적화
   pos_label: 1 (기본값, 생략 가능)

교차검증 Folds: 5
랜덤 샘플링 조합 수: 150개
총 Fits: 150 × 5 = 750

🚀 RandomizedSearchCV 시작!
Fitting 5 folds for each of 150 candidates, totalling 750 fits


[LightGBM] [Fatal] Check failed: (best_split_info.left_count) > (0) at /__w/1/s/lightgbm-python/src/treelearner/serial_tree_learner.cpp, line 852 .



[CV] END model__colsample_bytree=0.749816047538945, model__learning_rate=0.047782143788446224, model__max_depth=5, model__min_child_samples=12, model__min_child_weight=1e-05, model__n_estimators=320, model__num_leaves=38, model__reg_alpha=0.32544423647442644, model__reg_lambda=0.028205789513550128, model__subsample=0.8887995089067299; total time=   8.7s


[LightGBM] [Fatal] Check failed: (best_split_info.left_count) > (0) at /__w/1/s/lightgbm-python/src/treelearner/serial_tree_learner.cpp, line 852 .

[LightGBM] [Fatal] Check failed: (best_split_info.left_count) > (0) at /__w/1/s/lightgbm-python/src/treelearner/serial_tree_learner.cpp, line 852 .

[LightGBM] [Fatal] Check failed: (best_split_info.left_count) > (0) at /__w/1/s/lightgbm-python/src/treelearner/serial_tree_learner.cpp, line 852 .



[CV] END model__colsample_bytree=0.7035119926400067, model__learning_rate=0.03481350279592919, model__max_depth=4, model__min_child_samples=10, model__min_child_weight=0.0001, model__n_estimators=405, model__num_leaves=18, model__reg_alpha=0.09242722776276352, model__reg_lambda=0.4847923138822793, model__subsample=0.9100531293444458; total time=   8.9s
[CV] END model__colsample_bytree=0.8827429375390468, model__learning_rate=0.03780532256184443, model__max_depth=3, model__min_child_samples=9, model__min_child_weight=0.0001, model__n_estimators=530, model__num_leaves=55, model__reg_alpha=0.4574798377718904, model__reg_lambda=0.42501928889489965, model__subsample=0.7797802696552814; total time=   9.0s
[CV] END model__colsample_bytree=0.7035119926400067, model__learning_rate=0.03481350279592919, model__max_depth=4, model__min_child_samples=10, model__min_child_weight=0.0001, model__n_estimators=405, model__num_leaves=18, model__reg_alpha=0.09242722776276352, model__reg_lambda=0.4847923138

...

[CV] END model__colsample_bytree=0.836965827544817, model__learning_rate=0.007090268572399898, model__max_depth=5, model__min_child_samples=11, model__min_child_weight=1e-05, model__n_estimators=466, model__num_leaves=32, model__reg_alpha=0.4744427686266666, model__reg_lambda=0.4828160165372797, model__subsample=0.9233589392465844; total time=  19.7s


[LightGBM] [Fatal] Check failed: (best_split_info.left_count) > (0) at /__w/1/s/lightgbm-python/src/treelearner/serial_tree_learner.cpp, line 852 .



[CV] END model__colsample_bytree=0.9272059063689972, model__learning_rate=0.04373287624653545, model__max_depth=5, model__min_child_samples=13, model__min_child_weight=0.0001, model__n_estimators=976, model__num_leaves=65, model__reg_alpha=0.13470616689926074, model__reg_lambda=0.12206276112388709, model__subsample=0.6673164168691722; total time=   4.5s


[LightGBM] [Fatal] Check failed: (best_split_info.left_count) > (0) at /__w/1/s/lightgbm-python/src/treelearner/serial_tree_learner.cpp, line 852 .



[CV] END model__colsample_bytree=0.836965827544817, model__learning_rate=0.007090268572399898, model__max_depth=5, model__min_child_samples=11, model__min_child_weight=1e-05, model__n_estimators=466, model__num_leaves=32, model__reg_alpha=0.4744427686266666, model__reg_lambda=0.4828160165372797, model__subsample=0.9233589392465844; total time=  27.2s
[CV] END model__colsample_bytree=0.8827429375390468, model__learning_rate=0.03780532256184443, model__max_depth=3, model__min_child_samples=9, model__min_child_weight=0.0001, model__n_estimators=530, model__num_leaves=55, model__reg_alpha=0.4574798377718904, model__reg_lambda=0.42501928889489965, model__subsample=0.7797802696552814; total time=  33.2s
[CV] END model__colsample_bytree=0.8827429375390468, model__learning_rate=0.03780532256184443, model__max_depth=3, model__min_child_samples=9, model__min_child_weight=0.0001, model__n_estimators=530, model__num_leaves=55, model__reg_alpha=0.4574798377718904, model__reg_lambda=0.42501928889489

In [ ]:
models = [
    'best_lgbm_smote_random',
    'best_lgbm_smote_20251106_155837',
    'best_lgbm_recall_optimized_20251106_162548',
    'best_lgbm_recall_optimized_20251106_201626'
]

In [ ]:
import joblib
loaded_model = joblib.load(MODEL_DIR / 'best_lgbm_recall_optimized_20251106_201626.pkl')
predictions = loaded_model.predict(X_test)

In [ ]:
loaded_model

In [ ]:
# 테스트셋 성능 평가
y_pred = loaded_model.predict(X_test)
y_prob = loaded_model.predict_proba(X_test)[:, 1]

print("\nClassification Report (Test Set):")
print(classification_report(y_test, y_pred, digits=3))

# Balanced Accuracy
bal_acc = balanced_accuracy_score(y_test, y_pred)
print(f"\nBalanced Accuracy (Test Set): {bal_acc:.3f}")

# ROC-AUC (OvO, macro)
y_test_bin = label_binarize(y_test, classes=np.unique(y_train))
roc_auc_ovo = roc_auc_score(y_test_bin, y_prob, average='macro')
print(f"ROC-AUC (OvO, macro, Test Set): {roc_auc_ovo:.3f}")

In [ ]:
model = best_model.named_steps['model']

In [ ]:
import shap

# SHAP 적용
shap.initjs()
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_test)

# Plot
shap.summary_plot(shap_values, X_test)


In [ ]:
# mean absolute shap value per feature
shap_importance = np.abs(shap_values).mean(axis=0)
print(shap_importance)
shap_df = pd.DataFrame({
    'feature': X_test.columns,
    'shap_importance': shap_importance
}).sort_values(by='shap_importance', ascending=False)

print(shap_df)
